In [1]:
import pandas as pd
import re
import os

In [2]:
n = len(os.listdir())
n/2

251.0

In [3]:
userDir = %pwd
#CRISPResso = "docker run -v ./"   + " -w " +  userDir + " -i pinellolab/crispresso2 CRISPResso"
CRISPResso = "CRISPResso"
parameter = "  -w 10 --min_frequency_alleles_around_cut_to_plot 0.01 --window_around_sgrna 10 --plot_window_size 15 --exclude_bp_from_left 1 --exclude_bp_from_right 1"

In [4]:
# DNA序列工具
def reverseDNA(dna):
    #a = list(dna)
    result = ''
    dna=dna.upper().strip()
    b = ''.join(reversed(dna))
    for i in b:
        if i == "A":
            j = "T"
        elif i == "T":
            j = "A"
        elif i == "C":
            j = "G"
        elif i == "G":
            j = "C"
        else:
            j = "N"

        result = result + j
    return  result


In [5]:

ref = pd.read_excel("VIP-5new.xlsx")

In [6]:
ref

,Unnamed: 0,Unnamed: 1,Name,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,sgRNA1,sgRNA2,amplico-before,amplico-after
0,20220213-1,HR-UASg-FANCF-LdsDNA,NaN,i501,TATAGCCT,Index1,ATCACG,GGGGTCCCAGGTGCTGACGT,GGAATCCCTTCTGCAGCACC,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...
1,20220213-2,HR-UASg-RUNX1-LdsDNA,NaN,i502,ATAGAGGC,Index1,ATCACG,GCATTTTCAGGAGGAAGCGA,ATGAAGCACTGTGGGTACGA,ggggagaagaaagagagatgtagggctagaggggtgaggctgaaac...,ggggagaagaaagagagatgtagggctagaggggtgaggctgaaac...
2,20220213-3,5’-UASg-RUNX1,NaN,i503,CCTATCCT,Index1,ATCACG,GCATTTTCAGGAGGAAGCGA,ATGAAGCACTGTGGGTACGA,ggggagaagaaagagagatgtagggctagaggggtgaggctgaaac...,ggggagaagaaagagagatgtagggctagaggggtgaggctgaaac...
3,20220213-4,5'-UASg-FANCF,NaN,i504,GGCTCTGA,Index1,ATCACG,GGGGTCCCAGGTGCTGACGT,GGAATCCCTTCTGCAGCACC,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...
4,20220213-5,HR-LacO-FANCF-LdsDNA,NaN,i505,AGGCGAAG,Index1,ATCACG,GGGGTCCCAGGTGCTGACGT,GGAATCCCTTCTGCAGCACC,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...,GCAGCCGCCGCTCCAGAGCCGTGCGAATGGGGCCATGCCGACCAAA...
...,...,...,...,...,...,...,...,...,...,...,...
240,20220705-24,dsDNA-AsCas12f1-PDCD1-100bp-(-3)-TtoG,NaN,i501,TATAGCCT,index34,GCCATG,CTGTGAGCTCTAGTCCCCAC,NaN,GCCCTGACCCTGGCAGGCATATGTTTCAGGAGGTCCTTGTCTTGGG...,GCCCTGACCCTGGCAGGCATATGTTTCAGGAGGTCCTTGTCTTGGG...
241,20220705-25,dsDNA-AsCas12f1-PDCD1-100bp-(+1)-CtoT,NaN,i502,ATAGAGGC,index34,GCCATG,CTGTGAGCTCTAGTCCCCAC,NaN,GCCCTGACCCTGGCAGGCATATGTTTCAGGAGGTCCTTGTCTTGGG...,GCCCTGACCCTGGCAGGCATATGTTTCAGGAGGTCCTTGTCTTGGG...
242,20220705-26,dsDNA-AsCas12f1-TP51-1-100bp-(+1)-CtoA,NaN,i503,CCTATCCT,index34,GCCATG,GCTGGGGAGAGGAGCTGGTG,NaN,CACGAGCTGCCCCCAGGGAGCACTAAGCGAGGTAAGCAAGCAGGAC...,CACGAGCTGCCCCCAGGGAGCACTAAGCGAGGTAAGCAAGCAGGAC...
243,20220705-27,dsDNA-AsCas12f1-TP51-1-100bp-(+3)-CtoA,NaN,i504,GGCTCTGA,index34,GCCATG,GCTGGGGAGAGGAGCTGGTG,NaN,CACGAGCTGCCCCCAGGGAGCACTAAGCGAGGTAAGCAAGCAGGAC...,CACGAGCTGCCCCCAGGGAGCACTAAGCGAGGTAAGCAAGCAGGAC...


In [7]:
#生成批处理文件
bashData = ["#!/bin/bash\n date > timeCounter \n"] #bash文件头
authorInfo = """This Script is generated automatically. Do not modify anything unless you know what you are doing.
Script Author:\tMo Qiqin
Contact:\tmoqq@shanghaitech.edu.cn
"""
thread = 12
counter = 0

In [8]:
cmdFrame = pd.DataFrame(columns=["cmd"])

In [9]:
log = ''
cmdList = []
fileList = os.listdir()

for i in ref.index:
    sample = ref.loc[i]["Unnamed: 0"]
    sg1 = ref.loc[i]["sgRNA1"]
    sg2 = ref.loc[i]["sgRNA2"]
    seqPair = []
    
    
    #测序文件一一匹配
    for f in fileList:
        if sample+"_" in f:
            seqPair.append(f)
    if seqPair:
        r1 = seqPair[0]
        r2 = seqPair[1]
        if len(seqPair) != 2:
            print(sample)
            print(seqPair)
    else:
        print(sample + " not found")
        log = log + (sample + " not found")
        continue
    
    
    
    #sg序列读取
    if (str(sg2) != "nan") and ("nan" != str(sg1)):
        sg = sg1 + "," + sg2
        #print(sg)
    else:
        sg = str(sg1) + str(sg2)
        sg = sg.replace("nan","")
        #print(sg)
        
    #比对的目标序列
    amplicon = ref.loc[i]["amplico-before"]
    hdrRef = ref.loc[i]["amplico-after"]
    cmd =  CRISPResso + (" -r1 %s -r2 %s  -a %s -g %s  -e %s"%(r1,r2,amplicon,sg,hdrRef)) + parameter + " -o results/%s"%(sample)
    cmdList.append(cmd)
    cmdFrame.loc[sample,"cmd"] = cmd
    
    CMD ="{\n" + cmd + "\n}&\n" + "\n clear \n"
    bashData.append(CMD)
    counter = counter + 1
    if counter == thread:
        bashData.append("\nwait\n")
        counter = 0
    
        
    
    

In [10]:
bashData.append("\n wait \n date >> timeCounter\n")
a = "".join(bashData)
with open("run.sh","w") as f:
    f.write(a)

In [11]:
cmdFrame.to_excel("allCMD.xlsx")

In [12]:
cmdFrame

,cmd
20220213-1,CRISPResso -r1 20220213-1_S1_L001_R1_001.fastq...
20220213-2,CRISPResso -r1 20220213-2_S2_L001_R2_001.fastq...
20220213-3,CRISPResso -r1 20220213-3_S3_L001_R2_001.fastq...
20220213-4,CRISPResso -r1 20220213-4_S4_L001_R1_001.fastq...
20220213-5,CRISPResso -r1 20220213-5_S5_L001_R1_001.fastq...
...,...
20220705-24,CRISPResso -r1 20220705-24_S241_L001_R2_001.fa...
20220705-25,CRISPResso -r1 20220705-25_S242_L001_R1_001.fa...
20220705-26,CRISPResso -r1 20220705-26_S243_L001_R1_001.fa...
20220705-27,CRISPResso -r1 20220705-27_S244_L001_R1_001.fa...


In [13]:
ref = pd.read_excel("VIP-5.xlsx",index_col=0)
fileSheet = pd.DataFrame(columns=["描述","测序文件1","测序文件2"])
for i in cmdFrame.index:
    cmd = cmdFrame.loc[i,"cmd"]
    file = cmd.split("-a")[0].replace("CRISPResso -r1","")
    if "-r2" in file:
        file1  = file.split('-r2')[0].strip()
        file2 = file.split('-r2')[1].strip()
        fileSheet.loc[i,"描述"] = ref.loc[i,"Unnamed: 1"]
        fileSheet.loc[i,"测序文件1"] = file1
        fileSheet.loc[i,"测序文件2"] = file2
        
    

In [14]:
fileSheet

,描述,测序文件1,测序文件2
20220213-1,HR-UASg-FANCF-LdsDNA,20220213-1_S1_L001_R1_001.fastq.gz,20220213-1_S1_L001_R2_001.fastq.gz
20220213-2,HR-UASg-RUNX1-LdsDNA,20220213-2_S2_L001_R2_001.fastq.gz,20220213-2_S2_L001_R1_001.fastq.gz
20220213-3,5’-UASg-RUNX1,20220213-3_S3_L001_R2_001.fastq.gz,20220213-3_S3_L001_R1_001.fastq.gz
20220213-4,5'-UASg-FANCF,20220213-4_S4_L001_R1_001.fastq.gz,20220213-4_S4_L001_R2_001.fastq.gz
20220213-5,HR-LacO-FANCF-LdsDNA,20220213-5_S5_L001_R1_001.fastq.gz,20220213-5_S5_L001_R2_001.fastq.gz
...,...,...,...
20220705-24,dsDNA-AsCas12f1-PDCD1-100bp-(-3)-TtoG,20220705-24_S241_L001_R2_001.fastq.gz,20220705-24_S241_L001_R1_001.fastq.gz
20220705-25,dsDNA-AsCas12f1-PDCD1-100bp-(+1)-CtoT,20220705-25_S242_L001_R1_001.fastq.gz,20220705-25_S242_L001_R2_001.fastq.gz
20220705-26,dsDNA-AsCas12f1-TP51-1-100bp-(+1)-CtoA,20220705-26_S243_L001_R1_001.fastq.gz,20220705-26_S243_L001_R2_001.fastq.gz
20220705-27,dsDNA-AsCas12f1-TP51-1-100bp-(+3)-CtoA,20220705-27_S244_L001_R1_001.fastq.gz,20220705-27_S244_L001_R2_001.fastq.gz


In [15]:
fileSheet.to_excel("样品名-测序文件名对应关系表.xlsx","w")